# Gene essentiality

In [1]:
from __future__ import print_function, division, absolute_import

import sys

import qminospy
from qminospy.me2 import ME_NLP

# python imports
from copy import copy
import re
from os.path import join
from collections import defaultdict
import pickle

# third party imports
import pandas
import cobra
from tqdm import tqdm
import numpy as np
import scipy

# COBRAme
import cobrame
from cobrame.util import building, mu, me_model_interface
from cobrame.io.json import save_json_me_model, save_reduced_json_me_model

# ECOLIme
import ecolime
from ecolime import (transcription, translation, flat_files, generics, formulas, compartments)
from ecolime.util.helper_functions import *

%load_ext autoreload
%autoreload 2
print(cobra.__file__)
print(cobrame.__file__)
print(ecolime.__file__)

/home/jt/Downloads/cobrapy-0.5.11/cobra/io/sbml3.py:24: UserWarning: Install lxml for faster SBML I/O
  warn("Install lxml for faster SBML I/O")
/home/jt/Downloads/cobrapy-0.5.11/cobra/io/__init__.py:12: UserWarning: cobra.io.sbml requires libsbml
  warn("cobra.io.sbml requires libsbml")


/home/jt/Downloads/cobrapy-0.5.11/cobra/__init__.py
/home/jt/UCSD/cobrame-master/cobrame/__init__.py
/home/jt/UCSD/bacillusme-master/ecolime/__init__.py


In [2]:
eco_directory = join(flat_files.ecoli_files_dir, 'iJO1366.json')
ijo_directory = join(flat_files.ecoli_files_dir, 'iYO844.json')
uni_directory = join(flat_files.ecoli_files_dir, 'universal_model.json')

eco = cobra.io.load_json_model(eco_directory)
bsub = cobra.io.load_json_model(ijo_directory)
uni = cobra.io.load_json_model(uni_directory)

bsub.optimize()

<Solution 0.12 at 0x7fb649de92b0>

In [3]:
with open('../me_models/solution.pickle', 'rb') as solution:
    me = pickle.load(solution)

## Essentiality

M-model

In [4]:
from ecolime.analysis import essentiality as es

In [5]:
genes = [gene.id for gene in bsub.genes]
essentiality_df_m,flux_responses_m = es.gene_essentiality(bsub,genes, 'm',NP=10,threshold = 0.01,precision=1e-6)

10 threads: 100%|█████████▉| 843/844 [01:10<00:00, 11.96it/s]

Done


In [6]:
flux_responses_m.to_csv('essentiality_raw_results.csv')

ME-model

In [7]:
genes = [gene.id for gene in me.genes]
essentiality_df_me,flux_responses_me = es.gene_essentiality(me,genes,'me',NP=10,threshold = 0.01,precision=1e-6)

10 threads:  68%|██████▊   | 656/961 [25:03:09<10:56:53, 129.23s/it]

Done


In [13]:
flux_responses_me.to_csv('essentiality_raw_me_results.csv')

### Save combined results

In [14]:
essentiality_df = essentiality_df_m.join(essentiality_df_me,lsuffix='_m',rsuffix='_me')
essentiality_df = essentiality_df_m.join(essentiality_df[['response_me','essentiality_me']].replace(np.nan,'e'))

In [15]:
essentiality_df.to_csv('essentiality_results.csv')

In [16]:
essentiality_df[essentiality_df.essentiality != essentiality_df.essentiality_me]

,response,essentiality,response_me,essentiality_me
BSU22440,-1.000000e+00,e,0,0
BSU24350,-1.000000e+00,e,0,0
BSU29210,-1.000000e+00,e,0,0
BSU24340,-1.000000e+00,e,0,0
BSU29200,-1.000000e+00,e,0,0
BSU08090,0.000000e+00,0,e,e
BSU29690,0.000000e+00,0,e,e
BSU29710,0.000000e+00,0,e,e
BSU29700,0.000000e+00,0,e,e
BSU08080,0.000000e+00,0,e,e


In [17]:
essentiality_df

,response,essentiality,response_me,essentiality_me
BSU07840,-2.960606e-06,0,-9.37541e-06,0
BSU16730,-1.684547e-10,0,0,0
BSU16740,-1.684547e-10,0,0,0
BSU30820,-1.000000e+00,e,e,e
BSU06310,-3.015211e-10,0,0,0
...,...,...,...,...
BSU32470,0.000000e+00,0,0,0
BSU02860,0.000000e+00,0,0,0
BSU02850,0.000000e+00,0,0,0
BSU17610,-4.388287e-10,0,0,0
